# 6.4循环神经网络的从零开始实现

In [1]:
import torch
import torch.nn as nn
import time
import math
import sys
sys.path.append(".data/dive")
import d2lzh_pytorch as d2l
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 6.4.1one-hot向量

In [2]:
def one_hot(x, n_class, dtype=torch.float32):
    '''
    @x：一维向量，每个元素都是字符的索引
    @n_class：词典的大小vocab_size
    @dtype：返回向量类型
    '''
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)  # shape: (n, n_class)
    result.scatter_(1, x.long().view(-1, 1), 1)  
    # result[i, x[i, 0]] = 1，根据x把result的每一行对应的位置改为1
    # .scatter_(dim,index,src)：根据dim和index把src的元素重新分配
    # dim=1：tensor.scatter_(1,index,scr):scr[i,j]=output[i,index[i,j]]
    return result
    
x = torch.tensor([0, 2])
x_one_hot = one_hot(x, vocab_size)
print(x_one_hot)
print(x_one_hot.shape)
print(x_one_hot.sum(axis=1))

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])
torch.Size([2, 1027])
tensor([1., 1.])


In [3]:
def to_onehot(X, n_class):
    '''
    @X：每次采样的小批量，(batch_size,num_step)
    @n_class：字典大小
    '''
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]
    # 每次取出X的一列，调用ont_hot函数，得到形状是(batch_size,vocab_size)的二维矩阵
    # 列表长度为num_step

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1027])


## 6.4.2初始化模型参数

In [4]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
# num_inputs: d
# num_hiddens: h, 隐藏单元的个数是超参数
# num_outputs: q

def get_params():
    def _one(shape):
        param = torch.zeros(shape, device=device, dtype=torch.float32)
        nn.init.normal_(param, 0, 0.01)
        return torch.nn.Parameter(param)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device))
    return (W_xh, W_hh, b_h, W_hq, b_q)

## 6.4.3定义模型

In [5]:
def init_rnn_state(batch_size, num_hiddens, device):
    '''
    返回初始化的隐藏状态，元组
    @num_hiddens：隐藏单元个数，即h
    '''
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [6]:
def rnn(inputs, state, params):
    '''
    定义在一个时间步里如何计算隐藏状态和输出
    @inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    @state：维护的状态的初始值，元组
    @params：模型参数
    '''
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,) # H可以用在相邻采样

In [7]:
print(X.shape) # batch_size=2，num_step=5
print(num_hiddens)
print(vocab_size)
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params) # 前向计算
print(len(inputs), inputs[0].shape) # 列表，num_step；(batch_size,vocab_size)
print(len(outputs), outputs[0].shape) # 各个时间步的输出
print(len(state), state[0].shape) # 元组，1；(batch_size,num_hiddens)
print(len(state_new), state_new[0].shape) # 新的状态

torch.Size([2, 5])
256
1027
5 torch.Size([2, 1027])
5 torch.Size([2, 1027])
1 torch.Size([2, 256])
1 torch.Size([2, 256])


## 6.4.4定义预测函数

In [8]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    '''
    给定前缀prefix，预测接下来的num_chars个字符
    思路：处理prefix，H记录prefix的序列信息，处理prefix最后一个字符时
    已经对下一字符进行了预测，所以可以用预测的字符作为下一时刻的输入，
    重复，直至模型预测出num_chars个字符
    
    '''
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]   # output记录prefix+预测的num_chars个字符
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入，构造1*1的Tensor表示一个时间步
        # 调用to_onehot，得到长度为1的列表，元素形状(1，vocab_size)
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size) 
        # 计算输出和更新隐藏状态，Y：长度为1的列表
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y[0].argmax(dim=1).item()) # 预测字符的索引
    return ''.join([idx_to_char[i] for i in output])

In [9]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'分开感己烧绕灌晰草透拳提'

## 6.4.5裁剪梯度

In [10]:
def grad_clipping(params, theta, device):
    '''
    梯度裁剪，裁剪后的梯度不大于theta
    @params：模型参数
    @theta：预设阈值
    @device：设备
    '''
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum() # 记录梯度的L2范数
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

## 6.4.6困惑度

## 6.4.7定义模型训练函数

In [11]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
        # 相邻采样弊端：训练过程中，同一个epoch，随着batch的增大，损失函数关于隐藏变量的梯度传播得更远，计算开销也大
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  
                # 否则需要使用detach函数从计算图分离隐藏状态，这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列（防止
                # 梯度计算开销过大）
                for s in state:
                    s.detach_()
            # inputs是num_steps个形状为(batch_size, vocab_size)的矩阵
            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch*num_steps的向量，这样跟输出的行一一对应
            y = torch.flatten(Y.T)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清零
            if params[0].grad is not None: # 如不是第一个batch
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

## 6.4.8训练模型并创作歌词

In [12]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [ ]:
# 随机采样
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 72.282801, time 1.14 sec
 - 分开 我想要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开  一底我 别我的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我
epoch 100, perplexity 10.075586, time 1.13 sec
 - 分开 一场在 三步四 一九四步三在日步我 泪不休 语沉默 娘子却人在江日等我 泪不休 语沉默 娘子却人在
 - 不分开只 我想要再想 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我已好这节活
epoch 150, perplexity 2.848928, time 1.15 sec
 - 分开 一只两停留 谁底它停前的 为什么我女朋友场外加油 你却还让我出糗 可小就耳濡目染 什么刀枪跟棍棒 
 - 不分开吗 我不能再  我的脚我 我是你烦前 我将往事抽离 如果我遇见你是一场悲剧 我可以让生命就你 不小太


In [ ]:
# 相邻采样
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)